*Load packages and data:*

In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'0_GlobalData.xlsx'), read)
db.updateAlias(alias=[('BFt','BFt_alias')])
readSets(db)
db['mc'] = lpModels.mc(db)

# The ```lpBlock``` class

The ```lpBlock``` class is used to define linear programming (**LP**) models for the ```scipy.optimize.linprog``` method. The highlights of the class is:
1. Allows for symbols defined and constraints defined as ```pd.Series``` to be defined over sets ```pd.Index```.
2. Allows for the user to split up the **LP** problem in sub-problems; the ```lbBlock``` combines sub-problems to full matrices/vectors checking for missing domains etc.
3. Allows for fast implementation of complicated constraints by relying on ```alias``` methods for symbols and constraints (as used in *GAMS*).

## 1. Standard-form **LP** models:
<a id='StandardForm'>

The ```scipy.optimize.linprog``` solves the **LP** problem defined on the standard form:
$$\begin{align} \tag{1}
    &\min_{x}\mbox{ }c^T\cdot x \\ 
    &A_{ub}\times x \leq b_{ub} \\ 
    &A_{eq}\times x  = b_{eq} \\ 
    &l\leq x\leq u,
\end{align}$$
where:
* $c, x, l, u$ are vectors of length $N$,
* $b_{eq}, b_{ub}$ are vectors of lengths $N_{eq}, N_{ub}$, 
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

The solver takes inputs as ```np.arrays```, which means that we have to be careful with what order different variables and constraints are added to all symbols: The $n$'th element in $c$ represents the same element as the $n$'th elements in $l,u$ and the $n$'th column vectors in $A_{ub}, A_{eq}$. Thus, when solving a model with many different types of constraints and variables, the task of constructing suitable vectors / matrices becomes quite cumbersome. The main task of the ```lpBlock``` compiler is to make it easier to formulate this type of **LP** models.

### Model example
<a id='example'>

In the following, we will use the following simple linear model as example:
$$\begin{align} \tag{2}
    \min_{E_i, B_f} SC &= \sum_i^Nc_i E_i \\ 
                L &= \sum_i E_i \\ 
               B_f&= \sum_i\mu_f^i E_i, && \text{for }f=f_1,...,f_F\\
               E_i&\in[0,q_i], && \text{for }i=1,...,N. 
\end{align}$$

This system of equations represents minimizing the system costs $(SC)$ of satisfying a given level of demand $(L)$ by supply $(E_i)$ from $N$ different sources at the cost $c_i$. Each producer can at most produce $q_i$. In producing $E_i$ the producer $i$ uses $\mu_f^iE_i$ of fuel type $f$. The total use of fuel $B_f$ is computed by summing over all $i$ suppliers. 

We note that it is straightforward to leave out $B_f$ from the system and simply compute this having solved for $E_i$. Keeping the computation of $B_f$ in the simple helps highlighting features of the compiler.

The relevant parameters for the model example is pre-loaded to the database ```db```. The syntax is slightly different here though:
* Sets: $i$=```'id'``` and $f$ = ```'BFt'```.
* Variables: $E_i$ = ```'Generation'```, $B_f$ = ```'FuelConsumption'```.
* Parameters: $c_i$ = ```'mc'```, $L$=```'Load'```, $\mu_f^i$ = ```'FuelMix'```, $q_i$ = ```'GeneratingCapacity'```.

## 2. Inputs to the ```lpBlock```

The class builds on five main inputs corresponding to each of the blocks outlined in the standard form approach above: ```('c','eq','ub','l','u')```. The blocks should be added with a specific syntax that is easiest to showcase using the [model example](#example):

In [2]:
block = lpCompiler.lpBlock()

### 2.1. $c$-blocks

The $c$-block specifies the parameters $c^T$ in the [standard form](#StandardForm). In the model example, note that we solve for both $E$ and $B$. The coefficients are thus specified as:

In [3]:
c_E = db['mc'] # coefficients in c^T loaded onto vector of Ei's
c_B = pd.Series(0, index = db['BFt']) # coefficients in c^T loaded onto vector of Bf's.

and added to the ```lpBlock``` using the syntax: The ```'variableName'``` identifies the relevant variable coefficients are loaded onto and ```'parameter'``` the coefficients that we use.

In [4]:
block['c'] = [{'variableName': 'Generation', 'parameter': c_E}, 
              {'variableName': 'FuelConsumption', 'parameter': c_B}]

If we stopped here, the ```lpBlock``` would stack and sort the parameters, and return it in the appropriate np.array:

In [5]:
block()['c']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11.872520789065591,
       12.346413778840134, 12.193476419423797, 8.393216172041864,
       5.0310677452878725, 23.44365590801896, 3.0, 3.0], dtype=object)

#### Smart compiling:

Before we continue to the other types of blocks, let's highlight a couple of shortcuts that makes it easier to specify the model:
* Specifying ```self.globalDomains:``` In this example, the coefficients on ```FuelConsumption``` are 0 across the entire set ```BFt```. If we add this to ```self.globalDomains```, the compiler automatically infers the vector of coefficients on ```FuelConsumption```.
* Automatically filling in missing values: In this example, there are actually no coefficients needed on ```FuelConsumption``` to compute the objective function. One way to handle this is to tell the compiler that it should include a vector of zeros for ```FuelConsumption```. However, the compiler will automatically detect this, as long as the variable ```FuelConsumption``` is used in just one of the blocks. In this instance, the ```FuelConsumption``` variable is used in the equality constraint block ```'eq'```.

*Using ```self.globalDomains```:*

In [6]:
block.globalDomains['FuelConsumption'] = db['BFt'] 
block['c'] = [{'variableName': 'Generation', 'parameter': c_E}, 
              {'variableName': 'FuelConsumption', 'parameter': None}] # Note: We can simply input 0 or None as the parameter; the compiler automatically returns the correct vector of coefficients.
block()['c']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11.872520789065591,
       12.346413778840134, 12.193476419423797, 8.393216172041864,
       5.0310677452878725, 23.44365590801896, 3.0, 3.0], dtype=object)

*Automatic detection: This sets up the equality constraint that relies on 'FuelConsumption' (explanation of this syntax comes later in the notebook)*

In [7]:
block['c'] = [{'variableName': 'Generation', 'parameter': c_E}] # leave out the 'FuelConsumption' part entirely here; as long as it is used somewhere else, the compiler fills in zeros where it is needed.
block['eq'] = [
                {'constrName': 'eqFuelConsumption', 'b': pd.Series(0, index = db['BFt_alias']), 
                 'A': [{'variableName': 'Generation', 'parameter': rc_pd(db['FuelMix'], alias = {'BFt':'BFt_alias'})},
                       {'variableName': 'FuelConsumption', 'parameter': pd.Series(-1, index = pd.MultiIndex.from_arrays([db['BFt'], db['BFt_alias']]))}
                      ]
                }
               ]
block()['c']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11.872520789065591,
       12.346413778840134, 12.193476419423797, 8.393216172041864,
       5.0310677452878725, 23.44365590801896, 3.0, 3.0], dtype=object)

### 2.2. $u$ and $l$-blocks

The $u,l$-blocks specify the upper and lower bounds in the [standard form](#StandardForm). In the model example, the variable ```Generation``` has a lower bound of $0$ and an upper bound specified by the ```GeneratingCapacity``` variable. The variable ```FuelConsumption``` is not bounded. We specify this as follows:

In [8]:
l_Generation = pd.Series(0, index = db['id'])
u_Generation = db['GeneratingCapacity']
l_FuelConsumption = pd.Series(-np.inf, index = db['BFt'])
u_FuelConsumption = pd.Series(np.inf, index = db['BFt'])

and add this to the ```lpBlock``` using a syntax equivalent to the $c$-blocks:

In [9]:
block['l'] = [{'variableName': 'Generation', 'parameter': l_Generation}, 
              {'variableName': 'FuelConsumption', 'parameter': l_FuelConsumption}]
block['u'] = [{'variableName': 'Generation', 'parameter': u_Generation},
              {'variableName': 'FuelConsumption', 'parameter': u_FuelConsumption}]

```lpBlock``` stacks and sorts the parameters into the ```bounds``` parameter as a sequence of lower, upper bounds:

In [10]:
block()['bounds']

array([[-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [-inf,  inf],
       [  0.,  10.],
       [  0.,  15.],
       [  0.,  10.],
       [  0.,  30.],
       [  0.,   5.],
       [  0.,   8.],
       [  0.,  35.],
       [  0.,  10.]])

#### Smart compiling:

We can use the same methods as outlined for the $c$-block to minimize the information we parse to the compiler: 
* ```self.globalDomains:``` If the domain of a variable is specified in the ```self.globalDomains``` any scalar values are automatically broadcasted to these domains. In this example, we can specify ```-np.inf``` and ```np.inf``` as the bounds on 'FuelConsumption', if we have added ```BFt``` to the global domains. Note that if we pass the value ```None```, the compiler default to using ```0``` for lower bounds, and ```np.inf``` for upper bounds. 
* Automatically filling in missing values: In this example, we can drop the upper bounds for ```FuelConsumption``` and lower bounds for ```Generation``` as we are using default values anyway. As long as the variables ```FuelConsumption``` and ```Generation``` are used somewhere in the model (e.g. in the 'eq' or 'c') blocks, we don't need to provide any information where default values suffices. 

*Using ```self.globalDomains```:*

In [11]:
block.globalDomains.update({'FuelConsumption': db['BFt'], 'Generation': db['id']})
block['l'] = [{'variableName': 'Generation', 'parameter': None}, # using None means that the compiler defaults to 0 for the lower bound. 
              {'variableName': 'FuelConsumption', 'parameter': -np.inf}]
block['u'] = [{'variableName': 'Generation', 'parameter': u_Generation},
              {'variableName': 'FuelConsumption', 'parameter': None}] # Using None means that the compiler defaults to np.inf for the upper bound.
block()['bounds']

array([[-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [  0.,  10.],
       [  0.,  15.],
       [  0.,  10.],
       [  0.,  30.],
       [  0.,   5.],
       [  0.,   8.],
       [  0.,  35.],
       [  0.,  10.]])

*Automatic detection of missing values:*

In [12]:
block['l'] = [{'variableName': 'FuelConsumption', 'parameter': -np.inf}]
block['u'] = [{'variableName': 'Generation', 'parameter': u_Generation}]
block()['bounds']

array([[-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [  0.,  10.],
       [  0.,  15.],
       [  0.,  10.],
       [  0.,  30.],
       [  0.,   5.],
       [  0.,   8.],
       [  0.,  35.],
       [  0.,  10.]])

### 2.3. $eq$ and $ub$ blocks:

The final and most complicated blocks to add are the constraint blocks $eq$ (equality) and $ub$ (inequality, upper bound). To specify these, we need to add both specification of constants ($b_e, b_{ub}$) and the coefficient matrices $A_{eq}, A_{ub}$. In the model example used here, we only deal with equality constraints, but $ub$ constraints are handled equivalently. Note that, mathematically, we are setting up the system of equations:

$$\begin{align}
        \underbrace{\begin{pmatrix} 1, & \cdots & 1, & 0, & \cdots & 0 \\
        \mu_1^1, & \cdots & \mu_1^N & -1, & \cdots & 0 \\ 
        \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\ \mu_F^1, & \cdots & \mu_F^N, & 0, & \cdots & -1
        \end{pmatrix}}_{\equiv A_{eq}} \times \underbrace{\begin{pmatrix} E_1 \\ \vdots \\ E_N \\ B_1 \\ \vdots \\ B_F \end{pmatrix}}_{=x} =
        \underbrace{\begin{pmatrix} L \\ 0 \\ \vdots \\ 0\end{pmatrix}}_{b_{eq}} \tag{3}
\end{align}$$

We refer to the first constraint as the ```'equilibrium'``` constraint $\sum_i E_i = L$, and the next $F$ constraints as ```'eqFuelConsumption'``` constraint (note: Try not to use the exact same names as parameters/variables used in the model).

#### $b$ coefficients

For the ```equilibrium``` constraint there is a single constraint, namely the ```Load``` variable:

In [13]:
b_equilibrium = db['Load']

For the ```eqFuelConsumption``` constraint, however, the situation is slightly more complicated: The constraint is $0$ repeated for all fuel types (```BFt```). However, the index ```BFt``` is used to indicate the set of variables on the left-hand side in equation (3). To indicate that the **constraint set** is separate from the **variable set**, we use the *alias* for ```BFt``` denoted ```BFt_alias```; an alias contains the same set elements as the aliased set, but with its own identifier (name):

In [14]:
b_eqFuelConsumption = pd.Series(0, index = db['BFt_alias'])

We add the constraints using the syntax:

In [15]:
block = lpCompiler.lpBlock()
block['eq'] = [{'constrName': 'eqFuelConsumption', 'b': b_eqFuelConsumption},
               {'constrName': 'equilibrium', 'b': b_equilibrium}]

#### $A$ matrix coefficients

For the ```equilibrium``` constraint, we need to add the vector of ones for the variable ```Generation```, and zeros for ```FuelConsumption```:

In [16]:
A_equilibrium_Generation = pd.Series(1, index = db['id'])
A_equilibrium_FuelConsumption = pd.Series(0, index = db['BFt'])

We add the coefficients using the syntax:

In [17]:
block['eq'] += [{'constrName': 'equilibrium', 'A': [{'variableName': 'Generation', 'parameter': A_equilibrium_Generation},
                                                    {'variableName': 'FuelConsumption', 'parameter': A_equilibrium_FuelConsumption}]}]

For the ```eqFuelConsumption``` constraint, we add a matrix of ```FuelMix``` ($\mu_f^i$) for ```Generation``` and a diagonal matrix of $-1$ for ```FuelConsumption```. Importantly, recall that the ```Generation``` variable is only defined over ```id```, whereas the constraint is defined over the alias ```BFt_alias```. Thus, the matrix of fuel mix coefficients are defined over ```[BFt_alias, id]```:

In [18]:
A_eqFuelConsumption_Generation = rc_pd(db['FuelMix'], alias = {'BFt': 'BFt_alias'})

Similarly, to create the full matrix for the variable ```FuelConsumption```, we need to create a diagonal matrix of $-1$:

In [19]:
x = np.diag([-1]*len(db['BFt']))

and then stack it as a pandas series defined over ```[BFt, BFt_alias]```:

In [20]:
A_eqFuelConsumption_FuelConsumption = pd.Series(np.ndarray.flatten(np.stack(x)), index = pd.MultiIndex.from_product([db['BFt'], db['BFt_alias']]))

We add the coefficients using the syntax:

In [21]:
block['eq'] += [
                {'constrName': 'eqFuelConsumption', 'A': [{'variableName': 'Generation', 'parameter': A_eqFuelConsumption_Generation},
                                                          {'variableName': 'FuelConsumption', 'parameter': A_eqFuelConsumption_FuelConsumption}
                                                         ]
                }
               ]

The results are a matrix $A_{eq}$ and a vector $b_{eq}$ of dimensions:

In [22]:
block()['A_eq'].shape, block()['b_eq'].shape

((16, 23), (16,))

#### Smart compiling:

We can use the same methods as outlined for the other types of blocks, and a few extra ones, to minimize the information we give to the compiler
* ```self.globalDomains:``` If the domain of a variable is specified in the ```self.globalDomains``` any scalar values are automatically broadcasted to these domains. In this case, for instance, we can specify that the domain of the constraint ```'eqFuelConsumption'``` is ```BFt_alias```, and set ```b=None```.
* Automatically filling in missing values: In this example, we don't have to specify that the constraint ```eqFuelConsumption``` has a matrix of zeros for the variable ```FuelConsumption```. Similarly, we do not have to create a full diagonal matrix to specify how the variable ```Generation``` is used in the constraint ```eqFuelConsumption```, we just have to add the values that are non-zero.

Using the default values a bit more actively, we get the same results by specifying:

In [23]:
block = lpCompiler.lpBlock()
block.globalDomains.update({'eqFuelConsumption': db['BFt_alias'], 'Generation': db['id']})
block['eq'] = [{'constrName': 'eqFuelConsumption', 'b': None}, # use globalDomains to specify 'b' parameter
               {'constrName': 'equilibrium', 'b': db['Load']},
               {'constrName': 'eqFuelConsumption', 'A': [{'variableName': 'FuelConsumption', 'parameter': pd.Series(-1, index = pd.MultiIndex.from_arrays([db['BFt'], db['BFt_alias']]))},
                                                         {'variableName': 'Generation', 'parameter': rc_pd(db['FuelMix'], alias = {'BFt': 'BFt_alias'})}
                                                        ]},
               {'constrName': 'equilibrium', 'A': [{'variableName': 'Generation', 'parameter': 1}]}
              ]

The results are a matrix $A_{eq}$ and a vector $b_{eq}$ of dimensions:

In [24]:
block()['A_eq'].shape, block()['b_eq'].shape

((16, 23), (16,))

### 2.4. The full model

Specifying the model with minimal amount of information:

In [25]:
block = lpCompiler.lpBlock(globalDomains = {'Generation': db['id'], 'FuelConsumption': db['BFt'], 'eqFuelConsumption': db['BFt_alias']})
block['c'] = [{'variableName': 'Generation', 'parameter': db['mc']}] 
block['l'] = [{'variableName': 'FuelConsumption', 'parameter': -np.inf}]
block['u'] = [{'variableName': 'Generation', 'parameter': db['GeneratingCapacity']}]
block['eq']= [{'constrName': 'equilibrium', 'b': db['Load'], 'A': [{'variableName': 'Generation', 'parameter': 1}]},
              {'constrName': 'eqFuelConsumption', 'b': None, 'A': [{'variableName': 'Generation', 'parameter': rc_pd(db['FuelMix'], alias={'BFt':'BFt_alias'})},
                                                                   {'variableName': 'FuelConsumption', 'parameter': pd.Series(-1, index = pd.MultiIndex.from_arrays([db['BFt'], db['BFt_alias']]))}
                                                                  ]}
             ]

The ```self.__call__``` method compiles the model and returns arrays that can be used in the ```scipy.optimize.linprog``` solver:

In [26]:
sol = optimize.linprog(**block())

## 3. Repeated observations

As the previous section has outlined, all blocks are specified as ```lists``` (or iterators). Each element in the blocks is a dictionary that is either defined by a variable (```variableName```) or a constraint (```constrName```). If the same variable or constraint is added more than once, the following default behavior is used:
* In $c$-blocks: If the same variable is added more than one, the two entries are **added**. The idea is that the user can build the $c^T$ vector in the standard form as a sum of components.
* In $u,l$-blocks: If the same variable is constrained by more than one entry, the most stringent boundary is used. For the upper bound, this means that the compiler **only keeps the minimum**  and for the upper bound it **only keeps the maximum**.
* In $eq,ub$-blocks: Similar logic as $c$ blocks is applied. That means that $A,b$ can be constructed as sums of different inputs.

*Multiple $c$-entries: This adds 1 to all marginal costs in generation*

In [27]:
c_before = block()['c']
block['c'] = [{'variableName': 'Generation', 'parameter': db['mc']},
              {'variableName': 'Generation', 'parameter': 1}]
block()['c']-c_before #

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 1.0, 1.0, 1.0,
       1.0, 1.0, 1.0, 1.0], dtype=object)

*Multiple lower bound constraints: Set the minimum to 1 and then 2:*

In [28]:
block['l'] += [{'variableName': 'Generation', 'parameter': 1},
               {'variableName': 'Generation', 'parameter': 2}] # set the limit to 2, but only for the first two entries in 'id' set.
block()['bounds']

array([[-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [  2.,  10.],
       [  2.,  15.],
       [  2.,  10.],
       [  2.,  30.],
       [  2.,   5.],
       [  2.,   8.],
       [  2.,  35.],
       [  2.,  10.]])

*Multiple upper bound constraints: Set upper bound 50 for all. Then set it to 100. The effective limit will be 50:*

In [29]:
block['u'] = [{'variableName': 'Generation', 'parameter': 50},
              {'variableName': 'Generation', 'parameter': 100}] # set the limit to 2, but only for the first two entries in 'id' set.
block()['bounds']

array([[-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [-inf,  nan],
       [  2.,  50.],
       [  2.,  50.],
       [  2.,  50.],
       [  2.,  50.],
       [  2.,  50.],
       [  2.,  50.],
       [  2.,  50.],
       [  2.,  50.]])

Get indices for $x$-vector:

In [30]:
block.lp_solutionIndex

MultiIndex([('FuelConsumption',      'BioOil'),
            ('FuelConsumption',      'Biogas'),
            ('FuelConsumption',        'Coal'),
            ('FuelConsumption',     'Fueloil'),
            ('FuelConsumption',      'Gasoil'),
            ('FuelConsumption',    'Hydrogen'),
            ('FuelConsumption',     'Lignite'),
            ('FuelConsumption',      'NatGas'),
            ('FuelConsumption',        'Peat'),
            ('FuelConsumption',       'Straw'),
            ('FuelConsumption',     'Uranium'),
            ('FuelConsumption',       'Waste'),
            ('FuelConsumption',   'WoodChips'),
            ('FuelConsumption', 'WoodPellets'),
            ('FuelConsumption',   'WoodWaste'),
            (     'Generation',         'id1'),
            (     'Generation',         'id2'),
            (     'Generation',         'id3'),
            (     'Generation',         'id4'),
            (     'Generation',         'id5'),
            (     'Generation',         